In [1]:
#initial

import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from datetime import datetime
import math

words = pd.read_excel('dutch.xlsx',sheet_name='update')
words = words.loc[:,'word':]
lesson = pd.read_excel('dutch.xlsx',sheet_name='lesson')
lesson = lesson.loc[:,'lesson':]

In [49]:
#classes

class Words(object):
    def __init__(self, word, typ, translation, russian, example, example_translation, appear, trial_d, trial_r, success, weight):
        self.word = word
        self.typ = typ
        self.translation = translation
        self.example = example
        self.example_translation = example_translation
        self.russian = russian
        self.appear = appear
        self.trial_d = trial_d
        self.trial_r = trial_r
        self.success = success
        self.weight = weight
            
    def getTyp(self):
        return self.typ
    
    def getRussian(self):
        return self.russian
    
    def getExample(self):
        return self.example
    
    def changeType(self,new):
        self.typ = new
    
    def getTrials_d(self):
        return self.trial_d
    
    def getTrials_r(self):
        return self.trial_r
    
    def addTrials_d(self):
        self.trial_d += 1
        
    def addTrials_r(self):
        self.trial_r += 1
    
    def getAppear(self):
        return self.appear
    
    def addAppear(self):
        self.appear += 1
        return self.appear
       
    def addSuccess(self):
        self.success += 1
        return self.success   
    
    def getSuccess(self):
        return self.success
    
    def getWord(self):
        return self.word
    
    def getTranslation(self):
        return self.translation
    
    def getWeight(self):
        if self.trial_d == 0 and self.trial_r == 0:
            return self.weight
        else: 
            return 100 - (self.success/ (self.trial_d + self.trial_r)) * 100
        
    
    def __len__(self):
        return len(f'{self.word} -> {self.translation}')
    
    def __repr__(self):
        return f'{self.word}: {self.getTranslation()} , {self.getExample()}'
    
    def __str__(self):
        return f'{self.word} : {self.translation}'
    
    
class Lesson(object):
    def __init__(self, number):
        self.number = number
    def getNumber(self):
        return self.number
    def start(self, start):
        self.start = start
    def getStart(self):
        return self.start
    def inter(self, inter):
        self.inter = inter
    def getInter(self):
        return self.inter
    def finish(self, finish):
        self.finish = finish
    def getFinish(self):
        return self.finish
    def number_of_easy(self,easy):
        self.easy = easy
    def getNumber_of_easy(self):
        return self.easy
    def points(self, pts):
        self.pts = pts
    def add_pts(self, a_pts):
        self.pts += a_pts
    def getPoints(self):
        return self.pts + 1500 - int((self.finish - self.start).seconds)
    def wlist(self,list_of_words):
        self.list_of_words = list_of_words
    def getList(self):
        return self.list_of_words
    def length_of_lesson(self,lol):
        self.lol = lol
    def getLength_of_lesson(self):
        return self.lol

In [50]:
#defs

def loadWords(df_words, temp):
    wordList = []
    for i in range (len(words)):
        if temp == 'yes':
            wordList.append(Words(words.iloc[i,0],words.iloc[i,1],words.iloc[i,2],words.iloc[i,5], words.iloc[i,3],words.iloc[i,4],words.iloc[i,6],words.iloc[i,7],words.iloc[i,8],words.iloc[i,9],words.iloc[i,10]))
        else:
            wordList.append(Words(words.iloc[i,0],words.iloc[i,1],words.iloc[i,2],words.iloc[i,5], words.iloc[i,3],words.iloc[i,4],0,0,0,0, 100))       
    return wordList


def no_space(word):
    lst = []
    for w in word:
        lst.append(w.strip())
    return lst


def translation_with_comma(translation):
    try:
        translation = translation.split(',')
        translation = no_space(translation)
        return translation
    
    except:
        return [translation]
    
def help_for_guess(word, r): #r - number of letters
    list_of_word = list(word)
    copy = list_of_word.copy()
    for j in range(len(list_of_word)):
        if list_of_word[j] == ' ':
            copy.remove(list_of_word[j])
    
    l = random.sample(copy, k = r)
    
    for i in range(len(list_of_word)):
        if list_of_word[i] in l:
            list_of_word[i] = list_of_word[i]
        else:
            list_of_word[i] = '_'      
    back = ' '.join(list_of_word)
    
    return back


def rightword(word,translation,rever=0):
    point_counter = 0
    if rever == 1:
        b = 0
        while b == 0:
            x = input(f'[à ë ï é]\n\nPress "1","2","3" to open 1, 2, 3 letters in the word\n\n {translation}: ')
            if x == word:
                b = 1
                return [True, point_counter]
            elif x == '1' or x == '2' or x == '3':
                print(help_for_guess(word, int(x)))
                point_counter += int(x)
            else:
                return [False, point_counter]
                b = 1
    elif rever == 0:
        b = 0
        translation = translation_with_comma(translation)
        while b == 0:
            x = input(f'[à ë ï é]\n\nPress "1","2","3" to open 1, 2, 3 letters in the word\n\n {word}: ')
            if x in translation:
                b = 1
                return [True, point_counter]
            elif x == '1' or x == '2' or x == '3':
                print(help_for_guess(translation[0], int(x)))
                point_counter += int(x)
            else:
                b = 1
                return [False, point_counter]
    

def the_longest(sample_of_words):
    mx = 0
    for i in sample_of_words:
        if len(i) > mx:
            mx = len(i)
    return mx


def cycle(sample_of_words, rever):
    p = 250 
    s = sample_of_words.copy()
    while len(s) > 0:
        random.shuffle(s)
        lst_to_delete = []
        for i in s:
            temp = rightword(i.getWord(),i.getTranslation(),rever)
            if temp[0]:
                print("It's right!")
                p = p - temp[1]
                print(temp[1], p)
                i.addSuccess()
                if rever == 0:
                    i.addTrials_d()
                else:
                    i.addTrials_r()
                lst_to_delete.append(i)
            else:
                print("Shame on you!")
                p = p - temp[1]
                p -= 1
                print(p)
                if rever == 0:
                    i.addTrials_d()
                else:
                    i.addTrials_r()            
        if len(lst_to_delete) > 0:
            for w in lst_to_delete:
                s.remove(w)
            if len(s) != 0:
                plotting(s)
        else:
            if len(s) != 0:
                plotting(s)
            
            
    return p

def list_to_list(lst):
    st = ''
    for w in lst:
        st = st + w + '; '
    st.strip()       
    st = st[0:(len(st)-2)]
    return st

def lessonLenght(list_of_words):
    lessonLenght = 0
    for w in list_of_words:
        temp = w.getWord()
        temp = temp.replace(' ','')
        lessonLenght += len(temp)
    return lessonLenght

def random_sample(list_of_words, n):
    sample = random.choices(list_of_words, weights = [w.getWeight() for w in list_of_words], k = n )
    l = len(sample)
    st = 0
    count = 0
    while l != st:
        st = len(set(sample))
        sample = random.choices(list_of_words, weights = [w.getWeight() for w in list_of_words], k = n )
        count += 1
    print(count)
    
    return sample

In [51]:
#object creation
exist = 'yes'
wordList = loadWords(words,exist)
sample = random_sample(wordList,25)   

1


In [18]:
data_words_lesson_df = lesson
data_words_lesson_df = data_words_lesson_df.loc[:, ['points', 'r']]
data_words_lesson_df = data_words_lesson_df.sort_values(by='points', ascending=False, ignore_index=True)
mask = data_words_lesson_df.duplicated(subset='r', keep='first')
df = data_words_lesson_df[~mask]
df = df.reset_index(drop = True)
sml = df.nsmallest(5, columns = 'points')
sml.rename(columns={"points": "lesson/points"}, inplace = True)
sml.set_index("r", inplace = True)
#lst = sml['r'].to_list()
dct = sml.to_dict()
print('anti TOP5 lessons:', dct)

In [37]:
sml = df.nsmallest(5, columns = 'points')
sml.rename(columns={"points": "lesson/points"}, inplace = True)
sml.set_index("r", inplace = True)
#lst = sml['r'].to_list()
d = sml.to_dict()
print('anti TOP5 lessons:', d)

anti TOP5 lessons: {'lesson/points': {92: 619, 74: 835, 67: 842, 40: 853, 13: 925}}


In [25]:
print(sml['r'].to_list())

[92, 74, 67, 40, 13]


In [28]:
print('anti TOP5 lessons:', lst)

anti TOP5 lessons: [92, 74, 67, 40, 13]


In [30]:
sml.set_index()
sml.to_dict()

{'points': {104: 619, 103: 835, 102: 842, 101: 853, 100: 925},
 'r': {104: 92, 103: 74, 102: 67, 101: 40, 100: 13}}

In [38]:
lesson

,lesson,start,inter,finish,known,points,length,time,list_of_words,r
0,1,2022-11-26 21:14:16.069,2022-11-26 21:24:14.423,2022-11-26 21:31:48.654,6,877,184,1052,krapte; argument; jouw; achterwerk; broedstoof...,1
1,2,2022-11-27 02:30:31.842,2022-11-27 02:41:47.490,2022-11-27 02:49:34.282,8,803,185,1142,aanranden; fiks; middel; kroeshaar; plezant; a...,2
2,3,2022-11-27 13:24:02.594,2022-11-27 13:35:42.708,2022-11-27 13:42:12.905,8,865,204,1090,reis; betekenis; naast zitten; gewoon; voortaa...,3
3,4,2022-11-27 22:23:03.719,2022-11-27 22:34:28.615,2022-11-27 22:45:14.736,7,580,217,1331,recept; opzichtig; bloed; militair; verzuimen;...,4
4,5,2022-11-28 17:44:21.460,2022-11-28 17:52:57.397,2022-11-28 17:58:23.277,5,1115,191,841,Romeins; storten; half; alsnog; zuchten; rein;...,5
...,...,...,...,...,...,...,...,...,...,...
125,126,2023-01-18 20:16:05.869,2023-01-18 20:21:26.886,2023-01-18 20:27:49.923,12,1258,224,704,experiment; teleurstelling; vocht; kabouter; f...,100
126,127,2023-01-18 20:33:56.437,2023-01-18 20:39:47.719,2023-01-18 20:45:13.699,5,1275,225,677,het aan zijn water voelen; afwezigheid; toegan...,101
127,128,2023-01-19 20:54:06.259,2023-01-19 20:59:02.581,2023-01-19 21:02:48.158,11,1450,183,521,verschieten; erg; geluksvogel; Spaans; kwal; g...,102
128,129,2023-01-20 19:16:35.041,2023-01-20 19:21:52.401,2023-01-20 19:26:36.078,13,1356,205,601,ondergronds; verkouden zijn; industrieel; krui...,103
